# [Text To Speech] Synthetic Audio Dataset Creation
This sample demonstrates how to use Azure AI Speech API to generate synthetic audio dataset from text. 

> ✨ ***Note*** <br>
> Please check the supported languages and region availabilty before you get started - https://learn.microsoft.com/en-us/azure/ai-services/speech-service/language-support?tabs=tts / https://learn.microsoft.com/en-us/azure/ai-services/speech-service/regions  

## Prerequisites
Configure a Python virtual environment for 3.10 or later: 
 1. open the Command Palette (Ctrl+Shift+P).
 1. Search for Python: Create Environment.
 1. select Venv / Conda and choose where to create the new environment.
 1. Select the Python interpreter version. Create with version 3.10 or later.

## 1. Test Text to Speech Using the Speech SDK

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
import time
import json
import html
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

SPEECH_KEY = os.getenv("AZURE_AI_SPEECH_API_KEY")
SPEECH_REGION = os.getenv("AZURE_AI_SPEECH_REGION")
CUSTOM_SPEECH_LANG = os.getenv("CUSTOM_SPEECH_LANG")
CUSTOM_SPEECH_LOCALE = os.getenv("CUSTOM_SPEECH_LOCALE")
TTS_FOR_TRAIN = os.getenv("TTS_FOR_TRAIN")
TTS_FOR_EVAL = os.getenv("TTS_FOR_EVAL")

synthetic_text_file = ""
%store -r synthetic_text_file
try:
    synthetic_text_file
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the previous notebook again.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print(synthetic_text_file)

Create an instance of a speech config with specified subscription key and service region.
Replace with your own subscription key and service region (e.g., "westus").

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION)
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

In [ ]:
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

In [ ]:
print("Type some text that you want to speak...")
text = input()

### Test Speak text and Store the output stream to a wav file (speak_text_async)

In [ ]:
MIN_RETRIES = 2
for _ in range(MIN_RETRIES):
    try:
        result = speech_synthesizer.speak_text_async(text).get()
    except Exception as e:
        time.sleep(10)
        continue

In [ ]:
import os

output_dir = "sample"
    

if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized to speaker for text [{}]".format(text))
    stream = speechsdk.AudioDataStream(result)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    stream.save_to_wav_file(os.path.join(output_dir, "result_text.wav"))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
    print("Did you update the subscription info?")

### Test Speak SSML and Store the output stream to a wav file (speak_ssml_async)
- To get TTS voice id for specific languages, please check the voice gallery for more options, https://speech.microsoft.com/portal?projecttype=voicegallery

In [ ]:
default_tts_voice = 'en-US-JennyMultilingualV2Neural' # Default TTS voice for English, To get TTS voice id for specific languages, please check the voice gallery for more options
lang = "en-US"

ssml = f"""<speak version='1.0'  xmlns="https://www.w3.org/2001/10/synthesis" xml:lang='{lang}'>
                     <voice name='{default_tts_voice}'>
                             {html.escape(text)}
                     </voice>
                   </speak>"""

In [ ]:
if not os.path.exists(output_dir):
        os.makedirs(output_dir)
speech_sythesis_result = speech_synthesizer.speak_ssml_async(ssml).get()
stream = speechsdk.AudioDataStream(speech_sythesis_result)
stream.save_to_wav_file(os.path.join(output_dir,"result_ssml.wav"))

## 2. Generate syntethic dataset and manifest.txt file as a training dataset

### Set the ssml template for the dataset
- The ssml template is used to generate the synthetic dataset. Here is the reference for the ssml template: https://learn.microsoft.com/en-us/azure/ai-services/speech-service/speech-synthesis-markup

In [ ]:

def get_audio_file_by_speech_synthesis(text, file_path, lang, default_tts_voice):
    ssml = f"""<speak version='1.0'  xmlns="https://www.w3.org/2001/10/synthesis" xml:lang='{lang}'>
                     <voice name='{default_tts_voice}'>
                             {html.escape(text)}
                     </voice>
                   </speak>"""
    speech_sythesis_result = speech_synthesizer.speak_ssml_async(ssml).get()
    stream = speechsdk.AudioDataStream(speech_sythesis_result)
    stream.save_to_wav_file(file_path)

### Generate synthetic wav dataset and manifest for Specific language from the synthetic_text_file

In [ ]:
import datetime

languages = [CUSTOM_SPEECH_LOCALE] # List of languages to generate audio files
output_dir = "synthetic_data"
DELETE_OLD_DATA = True

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if(DELETE_OLD_DATA):
    for file in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, file))    

train_tts_voices = TTS_FOR_TRAIN.split(',')

for tts_voice in train_tts_voices:
    with open(synthetic_text_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                expression = json.loads(line)
                no = expression['no']
                for lang in languages:
                    text = expression[lang]
                    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                    file_name = f"{no}_{lang}_{timestamp}.wav"
                    get_audio_file_by_speech_synthesis(text, os.path.join(output_dir,file_name), lang, tts_voice)
                    with open(f'{output_dir}/manifest.txt', 'a', encoding='utf-8') as manifest_file:
                        manifest_file.write(f"{file_name}\t{text}\n")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}")
                print(e)



### Play WAV Files to test result in Output Folder
- Use the os library to list all WAV files in the output folder.

In [ ]:
import os
from IPython.display import Audio, display

files = os.listdir(output_dir)
wav_files = [file for file in files if file.endswith('.wav')]

# Sort wav_files by 'no' in ascending order
wav_files.sort(key=lambda x: int(x.split('_')[0]))
wav_files

### Play WAV Files
- Use IPython.display.Audio to play each WAV file listed in the output folder.

In [ ]:
# Play each WAV file in the output folder
for wav_file in wav_files[:3]:
    file_path = os.path.join(output_dir, wav_file)
    display(Audio(filename=file_path))

## 3. Create a Zip file for custom model training

- For audio + human - labeled data(Acoustic type) to train a custom speech model, you need to make a zip file of the audio files and the corresponding text file. 
- Here is an example of the structure of the labeled text file.

    > ```text
    > audio1.wav	Content like data, models, tests, and endpoints are organized into Projects in the Custom Speech portal. Each project is specific to a domain and country/language. For example, you may create a project for call centers that use English in the United States. To create your first project, select the Speech-to-text/Custom speech, then click New Project. Follow the instructions provided by the wizard to create your project. After you've created a project, you should see four tabs: Data, Testing, Training, and Deployment. Use the links provided in Next steps to learn how to use each tab.
    > audio2.wav	Custom Speech provides tools that allow you to visually inspect the recognition quality of a model by comparing audio data with the corresponding recognition result. From the Custom Speech portal, you can play back uploaded audio and determine if the provided recognition result is correct. This tool allows you to quickly inspect quality of Microsoft's baseline speech-to-text model or a trained custom model without having to transcribe any audio data.
    ```

In [ ]:
import zipfile
import shutil

DELETE_OLD_DATA = True

train_dataset_dir = "train_dataset"
if not os.path.exists(train_dataset_dir):
    os.makedirs(train_dataset_dir)

if(DELETE_OLD_DATA):
    for file in os.listdir(train_dataset_dir):
        os.remove(os.path.join(train_dataset_dir, file))    

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
zip_filename = f'train_{lang}_{timestamp}.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files:
        zipf.write(os.path.join(output_dir, file), file)

print(f"Created zip file: {zip_filename}")

shutil.move(zip_filename, os.path.join(train_dataset_dir, zip_filename))
print(f"Moved zip file to: {os.path.join(train_dataset_dir, zip_filename)}")
train_dataset_path = {os.path.join(train_dataset_dir, zip_filename)}
%store train_dataset_path

## 4. Generate Evaluation dataset for custom model evaluation

In [ ]:
import datetime

print(TTS_FOR_EVAL) 
languages = [CUSTOM_SPEECH_LOCALE]
eval_output_dir = "synthetic_eval_data"
DELETE_OLD_DATA = True

if not os.path.exists(eval_output_dir):
    os.makedirs(eval_output_dir)

if(DELETE_OLD_DATA):
    for file in os.listdir(eval_output_dir):
        os.remove(os.path.join(eval_output_dir, file))

eval_tts_voices = TTS_FOR_EVAL.split(',')

for tts_voice in eval_tts_voices:
    with open(synthetic_text_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                expression = json.loads(line)
                no = expression['no']
                for lang in languages:
                    text = expression[lang]
                    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                    file_name = f"{no}_{lang}_{timestamp}.wav"
                    get_audio_file_by_speech_synthesis(text, os.path.join(eval_output_dir,file_name), lang, tts_voice)
                    with open(f'{eval_output_dir}/manifest.txt', 'a', encoding='utf-8') as manifest_file:
                        manifest_file.write(f"{file_name}\t{text}\n")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {line}")
                print(e)

### Create a zip file for the evaluation dataset and move it to the eval_dataset directory

In [ ]:
import os
from IPython.display import Audio, display
import zipfile
import shutil

DELETE_OLD_DATA = True

eval_dataset_dir = "eval_dataset"
if not os.path.exists(eval_dataset_dir):
    os.makedirs(eval_dataset_dir)

if(DELETE_OLD_DATA):
    for file in os.listdir(eval_dataset_dir):
        os.remove(os.path.join(eval_dataset_dir, file))    

files = os.listdir(eval_output_dir)
wav_files = [file for file in files if file.endswith('.wav')]

# Sort wav_files by 'no' in ascending order
wav_files.sort(key=lambda x: int(x.split('_')[0]))

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
zip_filename = f'eval_{lang}_{timestamp}.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files:
        zipf.write(os.path.join(eval_output_dir, file), file)

print(f"Created zip file: {zip_filename}")



shutil.move(zip_filename, os.path.join(eval_dataset_dir, zip_filename))
print(f"Moved zip file to: {os.path.join(eval_dataset_dir, zip_filename)}")
eval_dataset_path = {os.path.join(eval_dataset_dir, zip_filename)}
%store eval_dataset_path